<h1 align="center"> Week 3 - RNN </h1>

<p style="text-align:center">汇报人：xxx</p>

---


###  建议学习资料：

- [李沐动手学深度学习54-59](https://space.bilibili.com/1567748478/lists/358497?type=series)


###  本周任务清单：

1.  **理论学习**
   - 学习循环神经网络（RNN）的基本原理  
   - 学习长短期记忆网络（LSTM）的基本原理  
   - 学习门控循环单元（GRU）的基本原理  
   - 对比 RNN、LSTM 与 GRU

2.  **实践任务**
   - 分别使用 RNN、LSTM 和 GRU 模型，在 RML2016 数据集上进行信号分类  
   - 比较三种模型（RNN / LSTM / GRU）在准确率、训练速度、收敛曲线等方面的表现  

3.  **提交要求**
   - 提交一个 `.ipynb` 文件，文件内容需包括：
     - 每个理论学习点下方填写的你整理的学习内容（可结合简单代码辅助理解）
     - 所有实践任务的结果展示（简单的实践任务代码可直接写在该文档）
   - 注：对文字排版无具体要求；使用 Jupyter Notebook 打开，图片可直接通过粘贴插入 Markdown  
   - 文件命名格式：`Week3_姓名.ipynb`


---

### 理论学习

#### 1. 循环神经网络（RNN）的基本原理  
- 掌握 RNN 的基本原理  
- 了解 RNN 存在的梯度消失与梯度爆炸问题  

#### 2. 长短期记忆网络（LSTM）的基本原理  
- 理解 LSTM 的核心结构：输入门、遗忘门、输出门  

#### 3. 门控循环单元（GRU）的基本原理  
- 理解 GRU 的核心结构：重置门与更新门
- 对比 GRU 与 LSTM 的结构

#### 4. RNN、LSTM 与 GRU 的对比  
- 对比三种结构在建模能力、训练速度和参数量上的差异  
- 分析三者在不同任务（如文本、语音）中的适用场景  
- 掌握如何根据任务需求选择合适的循环神经网络结构  

---

### 实践任务

#### 任务一：构建一个简单 RNN / LSTM / GRU 模型（用于调制识别）

**目标：** 使用 PyTorch 构建三个基础的循环神经网络模型（RNN、LSTM 和 GRU），对调制信号数据进行分类识别任务，并比较它们在该任务中的表现差异。

**要求：**

- 下载[RML2016.10a数据集](https://www.kaggle.com/datasets/raindrops12/rml201610a),使用提供的数据加载代码（输入数据维度为 `[batch_size, 2, 128]`，即2个通道、128个时间步）
- 分别构建以下三种模型结构：RNN 模型、LSTM 模型、GRU 模型。三者的整体结构一致，仅循环单元不同，具体结构如下（可自行修改，尝试提高准确率）：
  - 嵌入层（Embedding）：使用 `nn.Conv1d(2, 64, kernel_size=16, stride=8)` 
  - 循环层1：输入维度64，隐藏单元128
  - 循环层2：输入维度128，隐藏单元128
  - 分别使用 `nn.RNN`、`nn.LSTM` 和 `nn.GRU` 构建对应模型
  - 全连接层：输入128维，输出11类（使用序列的最后一个时间步的输出进行分类）
- 损失函数：`nn.CrossEntropyLoss()`
- 优化器：`Adam`
- 学习率：初始学习率设为 `0.001`，验证集准确率每3个epoch不下降，学习率x0.5
- EarlyStop：`patience=10`
- 可视化训练过程中的 `loss` 和 `accuracy` 曲线
- 输出模型在训练集与测试集上的分类准确率
- 对 RNN、LSTM、GRU 三种结构的运行时间、预测性能进行对比

In [27]:
import torch.utils.data as Data
import pandas as pd
import torch
import numpy as np

def to_num(y):
    label = []
    classes = ['8PSK','AM-DSB','AM-SSB','BPSK','CPFSK','GFSK','PAM4','QAM16','QAM64','QPSK', 'WBFM']
    for j in range(len(y)):
        label.append(classes.index(y[j]))
    label = torch.tensor(label)
    return label

def RML_dataloader(train_rate, valid_rate, test_rate, batch_size=64, random_seed=2024, num_workers=2):

    data = pd.read_pickle('RML2016.10a_dict.pkl')
    snrs, mods = map(lambda j: sorted(list(set(map(lambda x: x[j], data.keys())))), [1, 0])
    X = []
    label = []
    for mod in mods:
        for snr in snrs:
            X.append(data[(mod, snr)])
            for i in range(data[(mod, snr)].shape[0]):  label.append((mod, snr))
    X = np.vstack(X)

    n_examples = X.shape[0]
    n_train = int(train_rate * n_examples)

    train_idx = []
    test_idx = []
    val_idx = []

    np.random.seed(random_seed)
    Slices_list = np.linspace(0, n_examples, num=len(mods) * len(snrs) + 1)

    for k in range(0, Slices_list.shape[0] - 1):
        train_idx_subset = np.random.choice(
            range(int(Slices_list[k]), int(Slices_list[k + 1])), size=int(n_train / (len(mods) * len(snrs))),
            replace=False)
        Test_Val_idx_subset = list(set(range(int(Slices_list[k]), int(Slices_list[k + 1]))) - set(train_idx_subset))
        test_idx_subset = np.random.choice(Test_Val_idx_subset,
                                           size=int(
                                               (n_examples - n_train) * test_rate / (
                                                       (len(mods) * len(snrs)) * (test_rate + valid_rate))),
                                           replace=False)
        val_idx_subset = list(set(Test_Val_idx_subset) - set(test_idx_subset))

        train_idx = np.hstack([train_idx, train_idx_subset])
        val_idx = np.hstack([val_idx, val_idx_subset])
        test_idx = np.hstack([test_idx, test_idx_subset])

    train_indices = train_idx.astype('int64')
    val_indices = val_idx.astype('int64')
    test_indices = test_idx.astype('int64')

    X_train = torch.tensor(X[train_indices])  # [N, 2, 128]
    X_val = torch.tensor(X[val_indices])
    X_test = torch.tensor(X[test_indices])
    y_train, y_val, y_test = [], [], []
    snr_train, snr_val, snr_test = [], [], []

    for i in train_indices:
        y_train.append(label[i][0])
        snr_train.append(label[i][1])
    for i in val_indices:
        y_val.append(label[i][0])
        snr_val.append(label[i][1])
    for i in test_indices:
        y_test.append(label[i][0])
        snr_test.append(label[i][1])

    y_train = to_num(y_train)
    y_val = to_num(y_val)
    y_test = to_num(y_test)

    snr_test = torch.tensor(snr_test)
    snr_val = torch.tensor(snr_val)
    snr_train = torch.tensor(snr_train)

    torch_dataset = Data.TensorDataset(X_train, y_train, snr_train)
    loader1 = Data.DataLoader(
        dataset=torch_dataset,
        batch_size=batch_size,
        shuffle=True,
        num_workers=num_workers,
    )

    torch_dataset = Data.TensorDataset(X_val, y_val, snr_val)
    loader2 = Data.DataLoader(
        dataset=torch_dataset,
        batch_size=batch_size
        shuffle=True,
        num_workers=num_workers,
    )

    torch_dataset = Data.TensorDataset(X_test, y_test, snr_test)
    loader3 = Data.DataLoader(
        dataset=torch_dataset,
        batch_size=batch_size,
        shuffle=False,
        num_workers=num_workers,
    )

    return loader1, loader2, loader3  # train,valid,test

if __name__ == '__main__':
    train_loader, valid_loader, test_loader = RML_dataloader(0.6, 0.2, 0.2, batch_size=500, random_seed=2024)
